## Ejercicio 2: Implementar Monte-Carlo ES y Q-Learning

### Importamos las librerías

In [880]:
%matplotlib qt

import gymnasium
from gymnasium import spaces
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from collections import defaultdict

## Creación del entorno

### Sistemas de recompensas

Establecemos 3 sistemas distintos de recompensa:
- Recompensa simple por objetivos alcanzados.
- Recompensa de disatncia.
- Recompensa mixta (combinación de objetivos y distancias).

#### Simple

In [881]:
def calculate_reward_simple(env):
    """
    Calcula la recompensa basada únicamente en objetivos alcanzados y penalizaciones claras.

    Args:
        env (Environment): El entorno del robot (posiciones y estados).

    Returns:
        float: Recompensa acumulada según el estado actual.
    """
    reward = 0

    # Penalización por intentar recoger la pieza en una posición incorrecta.
    if env.has_piece == 0 and env.action_last == 6 and (
        env.robot_x_position != env.piece_x_position or
        env.robot_y_position != env.piece_y_position or
        env.robot_z_position != env.piece_z_position
    ):
        reward -= 50

    # Recompensa por estar donde la pieza.
    if env.robot_x_position == env.piece_x_position and env.robot_y_position == env.piece_y_position and env.robot_z_position == env.piece_z_position and env.has_piece == 0:
        reward += 10

    # Recompensa por recoger la pieza (solo una vez).
    if env.has_piece == 1 and not hasattr(env, 'piece_collected'):
        reward += 100
        setattr(env, 'piece_collected', True)

    # Penalización por abrir/cerrar pinza de forma repetitiva.
    if not hasattr(env, 'pinza_toggle_count'):
        env.pinza_toggle_count = 0

    if env.action_last == 6:
        env.pinza_toggle_count += 1
        if env.pinza_toggle_count > 2: # Penalizar solo si ocurre más de dos veces
            reward -= 500
    else:
        env.pinza_toggle_count = 0  # Reinicia si realiza otra acción

    # Recompensa por progresar hacia el objetivo en los ejes.
    if env.has_piece == 1:
        aligned_axes = sum([
            env.robot_x_position == env.goal_x_position,
            env.robot_y_position == env.goal_y_position,
            env.robot_z_position == env.goal_z_position
        ])
        reward += 50 * aligned_axes

    # Recompensa por estar alineado donde el objetivo mientras lleva la pieza.
    if env.has_piece == 1 and env.robot_x_position == env.goal_x_position and env.robot_y_position == env.goal_y_position and env.robot_z_position == env.goal_z_position:
        reward += 500

    # Recompensa por llevar la pieza al objetivo.
    if (env.piece_x_position == env.goal_x_position and
        env.piece_y_position == env.goal_y_position and
        env.piece_z_position == env.goal_z_position and
        not hasattr(env, 'goal_reached')):
        reward += 500
        setattr(env, 'goal_reached', True)

    # Recompensa por soltar la pieza en el objetivo.
    if env.has_piece == 0 and env.robot_x_position == env.goal_x_position and env.robot_y_position == env.goal_y_position and env.robot_z_position == env.goal_z_position:
        reward += 5000

    # Penalización por soltar la pieza fuera del objetivo.
    if env.has_piece == 0 and not hasattr(env, 'piece_placed') and (
        env.piece_x_position != env.goal_x_position or
        env.piece_y_position != env.goal_y_position or
        env.piece_z_position != env.goal_z_position
    ):
        reward -= 1000
        setattr(env, 'piece_placed', True)

    # Penalización adicional por soltar repetidamente la pieza en posiciones incorrectas.
    if not hasattr(env, 'incorrect_drop_count'):
        env.incorrect_drop_count = 0

    if env.has_piece == 0 and (
        env.piece_x_position != env.goal_x_position or
        env.piece_y_position != env.goal_y_position or
        env.piece_z_position != env.goal_z_position
    ):
        env.incorrect_drop_count += 1
        if env.incorrect_drop_count > 1:  # Penalizar solo si ocurre más de una vez
            reward -= 500 * env.incorrect_drop_count
    else:
        env.incorrect_drop_count = 0  # Reiniciar el contador si la pieza está en el lugar correcto

    # Recompensa constante por mantener la pieza.
    if env.has_piece == 1 and not (env.robot_x_position == env.goal_x_position and env.robot_y_position == env.goal_y_position and env.robot_z_position == env.goal_z_position):
        reward += 50

    # Penalización por movimientos redundantes.
    if hasattr(env, 'visited_positions'):
        current_position = (env.robot_x_position, env.robot_y_position, env.robot_z_position)
        if current_position in env.visited_positions:
            reward -= 50
        else:
            env.visited_positions.add(current_position)
    else:
        env.visited_positions = set([(env.robot_x_position, env.robot_y_position, env.robot_z_position)])

    # Penalización por entrar en bucles.
    if hasattr(env, 'recent_positions'):
        current_position = (env.robot_x_position, env.robot_y_position, env.robot_z_position)
        env.recent_positions.append(current_position)
        # Limitar el historial a las últimas 10 posiciones.
        if len(env.recent_positions) > 10:
            env.recent_positions.pop(0)
        # Penalizar bucles.
        if len(env.recent_positions) > len(set(env.recent_positions)):
            reward -= 200
    else:
        env.recent_positions = [(env.robot_x_position, env.robot_y_position, env.robot_z_position)]

    return reward


#### Distancia

In [882]:
def calculate_reward_distance(env):
    """
    Calcula la recompensa únicamente en función de las distancias al objetivo y la pieza.

    Args:
        env (Environment): El entorno que contiene el estado actual del robot. 

    Returns:
        float: La recompensa calculada para el estado actual del entorno.
    """
    reward = 0

    # Calcular la distancia al objeto
    distance_to_piece = ((env.robot_x_position - env.piece_x_position) ** 2 +
                         (env.robot_y_position - env.piece_y_position) ** 2 +
                         (env.robot_z_position - env.piece_z_position) ** 2) ** 0.5

    # Calcular la distancia al objetivo
    distance_to_goal = ((env.robot_x_position - env.goal_x_position) ** 2 +
                        (env.robot_y_position - env.goal_y_position) ** 2 +
                        (env.robot_z_position - env.goal_z_position) ** 2) ** 0.5

    if env.has_piece == 0:
        # Recompensa basada en la cercanía al objeto
        reward += max(0, 100 - distance_to_piece * 10)
    else:
        # Recompensa basada en la cercanía al objetivo
        reward += max(0, 500 - distance_to_goal * 10)

        # Recompensa adicional por reducir la distancia al objetivo
        if hasattr(env, 'previous_distance_to_goal'):
            if distance_to_goal < env.previous_distance_to_goal:
                reward += 50
        env.previous_distance_to_goal = distance_to_goal

    piece_distance_to_goal = ((env.piece_x_position - env.goal_x_position) ** 2 +
                            (env.piece_y_position - env.goal_y_position) ** 2 +
                            (env.piece_z_position - env.goal_z_position) ** 2) ** 0.5

    # Recompensa por llevar la pieza al objetivo
    if piece_distance_to_goal == 0 and env.has_piece == 0:
        reward += 100000

    return reward


#### Mixed

In [883]:

def calculate_reward_mixed(env):
    """
    Calcula la recompensa mixta basada en objetivos alcanzados y distancia al objetivo. Añade penalizaciones por movimientos redundantes y bucles.

    Args:
        env (Environment): El entorno que contiene el estado actual del robot.

    Returns:
        float: La recompensa calculada para el estado actual del entorno.
    """
    reward = 0 

    if env.has_piece == 0:
        # Recompensa proporcional a la cercanía al objeto.
        distance_to_piece = ((env.robot_x_position - env.piece_x_position) ** 2 +
                             (env.robot_y_position - env.piece_y_position) ** 2 +
                             (env.robot_z_position - env.piece_z_position) ** 2) ** 0.5
        reward += max(0, 50 - distance_to_piece * 2)
    else:
        # Recompensa proporcional a la cercanía al objetivo.
        distance_to_goal = ((env.robot_x_position - env.goal_x_position) ** 2 +
                            (env.robot_y_position - env.goal_y_position) ** 2 +
                            (env.robot_z_position - env.goal_z_position) ** 2) ** 0.5
        reward += max(0, 200 - distance_to_goal * 2)

        # Penalización y recompensa acumulativa basada en la distancia al objetivo.
        if hasattr(env, 'previous_distance_to_goal'):
            if distance_to_goal > env.previous_distance_to_goal:
                reward -= 5000
            else:
                reward += 10000
        env.previous_distance_to_goal = distance_to_goal

    # Recompensa por llegar a la pieza.
    if env.piece_x_position == env.robot_x_position and env.piece_y_position == env.robot_y_position and env.piece_z_position == env.robot_z_position:
        reward += 5000

    # Recompensa por recoger la pieza (solo una vez).
    if env.has_piece == 1 and not hasattr(env, 'piece_collected'):
        reward += 100
        setattr(env, 'piece_collected', True)

    # Recompensa constante por mantener la pieza.
    if env.has_piece == 1:
        reward += 10000

    # Recompensa por llegar al objetivo con la pieza.
    if env.piece_x_position == env.robot_x_position == env.goal_x_position and env.piece_y_position == env.robot_y_position == env.goal_y_position and env.piece_z_position == env.robot_z_position == env.goal_z_position:
        reward += 500000

    # Recompensa por alcanzar el objetivo.
    if (env.piece_x_position == env.goal_x_position and
        env.piece_y_position == env.goal_y_position and
        env.piece_z_position == env.goal_z_position and
        not hasattr(env, 'goal_reached')):
        reward += 1000000
        setattr(env, 'goal_reached', True)

    # Penalización por soltar la pieza en una posición incorrecta.
    if env.has_piece == 0 and not hasattr(env, 'piece_placed') and (
        env.piece_x_position != env.goal_x_position or
        env.piece_y_position != env.goal_y_position or
        env.piece_z_position != env.goal_z_position
    ):
        reward -= 10000
        setattr(env, 'piece_placed', True)

    # Penalización por movimientos redundantes.
    if hasattr(env, 'visited_positions'):
        current_position = (env.robot_x_position, env.robot_y_position, env.robot_z_position)
        if current_position in env.visited_positions:
            reward -= 50
        else:
            env.visited_positions.add(current_position)
    else:
        env.visited_positions = set([(env.robot_x_position, env.robot_y_position, env.robot_z_position)])

    # Penalización por entrar en bucles.
    if hasattr(env, 'recent_positions'):
        current_position = (env.robot_x_position, env.robot_y_position, env.robot_z_position)
        env.recent_positions.append(current_position)
        # Limitar el historial a las últimas 10 posiciones.
        if len(env.recent_positions) > 10:
            env.recent_positions.pop(0)
        # Penaliar bucles.
        if len(env.recent_positions) > len(set(env.recent_positions)):
            reward -= 20000
    else:
        env.recent_positions = [(env.robot_x_position, env.robot_y_position, env.robot_z_position)]

    return reward

### Definición del entorno

In [884]:
class RobotPickAndPlaceEnv(gymnasium.Env):
    """
    Representa el entorno del robot para resolver el problema de pick & place.

    Este entorno simula un espacio discreto en 3D donde un robot debe recoger una 
    pieza en un punto inicial y transportarla a un objetivo. El robot puede moverse 
    en las direcciones X, Y y Z, además de abrir o cerrar su pinza para coger y soltar la pieza.

    Attributes:
        robot_position: Posición actual del robot en (x, y, z).
        piece_position: Posición actual de la pieza en (x, y, z). (-1, -1, -1 si está siendo transportada).
        goal_position: Posición del objetivo en (x, y, z).
        has_piece (bool): Indica si el robot tiene la pieza agarrada (1 si tiene la pieza, 0 en caso contrario).
        step_limit (int): Número máximo de pasos permitidos.
        action_space (gymnasium.spaces.Discrete): Espacio de acciones (0-6).
        observation_space (gymnasium.spaces.Tuple): Espacio de observaciones del entorno.
        reward_function: Función de recompensa utilizada para evaluar los pasos del robot.
    """
    def __init__(self, reward_function=calculate_reward_simple):
        """
        Inicializa el entorno del robot y establece la función de recompensa, por defecto simple.

        Configura los espacios de acción y observación, y define las posiciones iniciales 
        del robot, la pieza y el objetivo.
        """
        super(RobotPickAndPlaceEnv, self).__init__()

        # Definir espacio de acciones: 0 = mover izquierda, 1 = mover derecha, 2 = mover abajo, 3 = mover arriba, 4 = mover atrás, 5 = mover delante, 6 = abrir/cerrar pinza
        self.action_space = spaces.Discrete(7)

        # Definir espacio de observaciones
        self.observation_space = spaces.Tuple((
            spaces.Discrete(10),  # Posición del robot en el eje X (0-9)
            spaces.Discrete(10),  # Posición del robot en el eje Y (0-9)
            spaces.Discrete(10),  # Posición del robot en el eje Z (0-9)
            spaces.Discrete(2),   # Tiene pieza (0 o 1)
            spaces.Discrete(11),  # Posición de la pieza en el eje X (0-9) (-1 si está siendo transportada)
            spaces.Discrete(11),  # Posición de la pieza en el eje Y (0-9) (-1 si está siendo transportada)
            spaces.Discrete(11)   # Posición de la pieza en el eje Z (0-9) (-1 si está siendo transportada)
        ))

        # Establecer función de recompensa
        self.reward_function = reward_function
        
        # Inicializar estado
        self.reset()

        # Inicializar figura, eje y gráfica de matplotlib
        self.fig = None
        self.ax = None

    def reset(self):
        """
        Reinicia el entorno a su estado inicial.

        Returns:
            tuple: Observación inicial del entorno.
        """
        # Condiciones iniciales
        self.robot_x_position = 0  # El robot comienza en la posición (0,0,0)
        self.robot_y_position = 0
        self.robot_z_position = 0
        self.has_piece = 0         # El robot no tiene la pieza
        self.piece_x_position = 5  # La pieza comienza en la posición (5,5,5)
        self.piece_y_position = 5
        self.piece_z_position = 5
        self.goal_x_position = 9   # El objetivo está en la posición (9, 7, 9)
        self.goal_y_position = 7
        self.goal_z_position = 9
        self.steps = 0             # Contador de pasos por episodio

        return self._get_observation()

    def step(self, action):
        """
        Realiza un paso en el entorno según la acción proporcionada.

        Args:
            action (int): Índice de la acción a realizar (0-6).

        Returns:
            tuple: Estado actual del entorno, recompensa obtenida, 
                indicador de si la tarea ha terminado, indicador de truncado, 
                y diccionario de información adicional.
        """
        # Asegurarse de que la acción es válida
        assert self.action_space.contains(action), "Acción inválida"
        done = False
        truncated = False
        self.action_last = action

        # Realizar movimientos según la acción
        if action == 0:  # Mover izquierda
            self.robot_x_position = max(0, self.robot_x_position - 1)
        elif action == 1:  # Mover derecha
            self.robot_x_position = min(9, self.robot_x_position + 1)
        elif action == 2:  # Mover abajo
            self.robot_y_position = max(0, self.robot_y_position - 1)
        elif action == 3:  # Mover arriba
            self.robot_y_position = min(9, self.robot_y_position + 1)
        elif action == 4:  # Mover atrás
            self.robot_z_position = max(0, self.robot_z_position - 1)
        elif action == 5:  # Mover adelante
            self.robot_z_position = min(9, self.robot_z_position + 1)
        elif action == 6:  # Abrir/cerrar pinza
            if not self.has_piece:
                if (self.robot_x_position == self.piece_x_position and
                    self.robot_y_position == self.piece_y_position and
                    self.robot_z_position == self.piece_z_position):
                    self.has_piece = 1
                    self.piece_x_position = -1
                    self.piece_y_position = -1
                    self.piece_z_position = -1
            else:
                self.has_piece = 0
                self.piece_x_position = self.robot_x_position
                self.piece_y_position = self.robot_y_position
                self.piece_z_position = self.robot_z_position
                if (self.piece_x_position == self.goal_x_position and
                    self.piece_y_position == self.goal_y_position and
                    self.piece_z_position == self.goal_z_position):
                    done = True

        # Calcular recompensa y verificar si se ha alcanzado el objetivo
        reward = self.reward_function(self)
        self.steps += 1
        
        if self.steps >= 1000:  # Límite de pasos por episodio
            truncated = True
        return self._get_observation(), reward, done, truncated, {}

    def render(self):
        """
        Renderiza el estado actual del entorno en una gráfica 3D.
        """
        # Inicializar figura y ejes si no existen
        if self.fig is None or self.ax is None:
            self.fig = plt.figure()
            self.ax = self.fig.add_subplot(111, projection="3d")

            # Inicializar gráficos del robot, la pieza y el objetivo
            self.robot_plot, = self.ax.plot([], [], [], "go", label="Robot")  # Punto verde para el robot
            self.piece_plot, = self.ax.plot([], [], [], "bo", label="Pieza")  # Punto azul para la pieza
            self.goal_plot, = self.ax.plot([], [], [], "ro", label="Objetivo")  # Punto rojo para el objetivo

            # Configuración de los límites del eje (ajústalos según tu entorno)
            self.ax.set_xlim([0, 10])
            self.ax.set_ylim([0, 10])
            self.ax.set_zlim([0, 10])

            # Etiquetas y leyenda
            self.ax.set_xlabel("X")
            self.ax.set_ylabel("Y")
            self.ax.set_zlabel("Z")
            self.ax.legend()

        # Actualizar posición del robot
        self.robot_plot.set_data([self.robot_x_position], [self.robot_y_position])
        self.robot_plot.set_3d_properties([self.robot_z_position])

        # Actualizar posición de la pieza
        if self.has_piece:
            self.piece_plot.set_data([self.robot_x_position], [self.robot_y_position])
            self.piece_plot.set_3d_properties([self.robot_z_position])
        else:
            self.piece_plot.set_data([self.piece_x_position], [self.piece_y_position])
            self.piece_plot.set_3d_properties([self.piece_z_position])

        # Actualizar posición del objetivo
        self.goal_plot.set_data([self.goal_x_position], [self.goal_y_position])
        self.goal_plot.set_3d_properties([self.goal_z_position])

        # Dibujar y pausar para visualización en tiempo real
        plt.draw()
        plt.pause(0.01)


    def _get_observation(self):
        """
        Obtiene la observación actual del entorno.

        Returns:
            tuple: Estado actual del entorno.
        """
        return (self.robot_x_position, self.robot_y_position, self.robot_z_position, self.has_piece, self.piece_x_position, self.piece_y_position, self.piece_z_position)
    
    def close(self):
        """
        Cierra la visualización del entorno.
        """
        plt.close()

## Implemetación de Algoritmos

### Monte-Carlo with Exploring Starts

In [885]:
def monte_carlo_es(env, num_episodes, gamma=0.9, eval_interval=1000, graph=False):
    """
    Implementa el método de Monte Carlo con Exploring Starts para resolver el entorno del robot.

    Args:
        env: Entorno del robot.
        num_episodes: Número de episodios a simular.
        gamma: Factor de descuento.
        eval_interval: Episodios entre evaluaciones de la política.
        graph: Booleano para mostrar la gráfica de recompensas acumuladas.

    Returns:
        Q: Función de valor óptima.
        policy: Política óptima.
    """
    Q = defaultdict(lambda: np.zeros(env.action_space.n))  # Inicializar Q
    returns = defaultdict(list)  # Retornos acumulados
    eval_results = {}  # Diccionario para almacenar métricas de evaluación
    accumulated_rewards = []  # Lista para almacenar recompensas acumuladas por episodio

    for episode in range(num_episodes):
        # Paso 1: Exploring Starts - Estado y acción aleatorios
        state = env.reset()[0]
        action = env.action_space.sample()

        # Paso 2: Generar un episodio completo
        episode_data = []
        done = False
        truncated = False

        while not (done or truncated):
            next_state, reward, done, truncated, _ = env.step(action)
            episode_data.append((state, action, reward))
            state = next_state
            action = env.action_space.sample()  # Exploración aleatoria

        # Paso 3: Calcular los retornos para cada par (estado, acción)
        G = 0
        visited = set()
        for state, action, reward in reversed(episode_data):
            G = reward + gamma * G
            if (state, action) not in visited:
                returns[(state, action)].append(G)
                Q[state][action] = np.mean(returns[(state, action)])
                visited.add((state, action))

        # Almacenar recompensa acumulada para este episodio
        accumulated_rewards.append(G)

        # Evaluar la política cada 'eval_interval' episodios
        if (episode + 1) % eval_interval == 0:
            policy = {state: np.argmax(actions) for state, actions in Q.items()}
            eval_results = evaluate_policy_env(env, policy, num_episodes=10, episode=episode + 1, eval_results=eval_results)

    if graph:
        # Generar la gráfica de episodios vs recompensa acumulada y mostrarla
        plt.figure(figsize=(10, 6))
        plt.plot(range(num_episodes), accumulated_rewards, label="Recompensa Acumulada")
        plt.xlabel("Episodios")
        plt.ylabel("Recompensa Acumulada")
        plt.title(f"Episodios vs Recompensa Acumulada (Monte Carlo) ({env.reward_function.__name__})")
        plt.legend()
        plt.grid()
        plt.savefig(f"/Users/maria/Desktop/Máster IA/Aprendizaje por Refuerzo/Clase 2/Ejercicio 2/monte_carlo_accumulated_rewards_{env.reward_function.__name__}.png")
        plt.show()

    # Derivar la política óptima de Q
    policy = {state: np.argmax(actions) for state, actions in Q.items()}

    # Mostrar gráficas finales de evaluación
    print("\nMostrando gráficas finales de evaluación...")
    plot_evaluation_results(eval_results)

    return Q, policy

Análisis del algoritmo Monte Carlo con Exploring Starts (ES)

1. **Exploración (Exploring Starts)**
- En cada episodio, el robot comienza desde un estado y una acción seleccionados de forma aleatoria. Asegura que todas las combinaciones de estados y acciones sean exploradas en algún momento, incluso en problemas con espacios de estados grandes o complejos.

2. **Aprendizaje (retornos acumulados)**
- Los valores `Q` se ajustan calculando el promedio de todos los retornos acumulados observados para cada par (estado, acción).

3. **Derivación de la política**
- Una vez calculados los valores `Q`, se deriva una política determinista seleccionando la acción con el mayor valor `Q` en cada estado.

### Q-Learning

In [886]:
def q_learning(env, num_episodes, alpha=0.1, gamma=0.9, epsilon=1.0, eval_interval=1000, graph=False):
    """
    Implementación del algoritmo Q-Learning.

    Args:
        env: Entorno del robot.
        num_episodes: Número de episodios a simular.
        alpha: Tasa de aprendizaje.
        gamma: Factor de descuento.
        epsilon: Tasa de exploración inicial.
        eval_interval: Episodios entre evaluaciones de la política.
        graph: Booleano para mostrar la gráfica de recompensas acumuladas.

    Returns:
        Q: Función de valor óptima.
        policy: Política óptima.
    """
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    eval_results = {}  # Diccionario para acumular evaluaciones
    accumulated_rewards = []  # Lista para almacenar recompensas acumuladas por episodio

    for episode in range(num_episodes):
        epsilon_end = 0.05
        epsilon_dec = max(epsilon_end, epsilon - (epsilon - epsilon_end) * episode / num_episodes) # Decae gradualmente el valor de epsilon para reducir la exploración a medida que el agente aprende.
        state = env.reset()[0]
        done = False
        truncated = False
        episode_reward = 0

        while not (done or truncated):
            # Selección de acción según la estrategia epsilon-greedy.
            if np.random.rand() < epsilon_dec:
                action = env.action_space.sample()
            else:
                action = np.argmax(Q[state])

            next_state, reward, done, truncated, _ = env.step(action)
            best_next_action = np.argmax(Q[next_state])
            
            # Actualización de la función Q usando la ecuación de Q-Learning.
            Q[state][action] += alpha * (reward + gamma * Q[next_state][best_next_action] - Q[state][action])
            state = next_state
            episode_reward += reward

        # Almacenar recompensa acumulada para este episodio
        accumulated_rewards.append(episode_reward)

        # Evaluar la política cada 'eval_interval' episodios
        if (episode + 1) % eval_interval == 0:
            print(f"\nEvaluación tras el episodio {episode + 1}:")
            policy = {state: np.argmax(actions) for state, actions in Q.items()}
            eval_results = evaluate_policy_env(env, policy, num_episodes=10, episode=episode + 1, eval_results=eval_results)

        if (episode + 1) % 1000 == 0:
            print(f"Episodio {episode + 1}/{num_episodes} completado")

    if graph:
    # Generar la gráfica de episodios vs recompensa acumulada y mostrarla
        plt.figure(figsize=(10, 6))
        plt.plot(range(num_episodes), accumulated_rewards, label="Recompensa Acumulada")
        plt.xlabel("Episodios")
        plt.ylabel("Recompensa Acumulada")
        plt.title(f"Episodios vs Recompensa Acumulada (Q-Learning) ({env.reward_function.__name__})")
        plt.legend()
        plt.grid()
        plt.savefig(f"/Users/maria/Desktop/Máster IA/Aprendizaje por Refuerzo/Clase 2/Ejercicio 2/q_learning_accumulated_rewards_{env.reward_function.__name__}.png")
        plt.show()

    # Derivar la política óptima de Q
    policy = {state: np.argmax(actions) for state, actions in Q.items()}

    # Mostrar gráficas finales de evaluación
    print("\nMostrando gráficas finales de evaluación...")
    plot_evaluation_results(eval_results)

    return Q, policy


Análisis detallado del algoritmo Q-Learning:
1. **Exploración (epsilon-greedy)**:
- En cada paso, el robot tiene una probabilidad epsilon de explorar tomando una acción aleatoria.
- En el resto de los casos, el robot explota su conocimiento seleccionando la acción con el mayor valor Q.

2. **Aprendizaje (actualización con alpha)**:
- En la actualización de Q utiliza una tasa de aprendizaje (alpha) para ajustar gradualmente los valores Q hacia los retornos esperados. Así, las acciones con mayores recompensas futuras se valoren más.

3. **Derivación de la política**:
- Después del entrenamiento, la política óptima se construye seleccionando la acción con el mayor valor Q en cada estado.

### Política de Evaluación

In [887]:
def evaluate_policy_env(env, policy, num_episodes=100, episode=None, eval_results={}):
    """
    Calcula las métricas de evaluación de una política en el entorno.

    Args:
        env: Entorno de simulación.
        policy (dict): Política a evaluar.
        num_episodes (int): Número de episodios para la evaluación.
        episode (int): Número de episodio actual (opcional, para referencia).
        eval_results (dict): Diccionario para acumular resultados de evaluaciones (opcional).

    Returns:
        dict: Diccionario con métricas acumuladas si eval_results es proporcionado.
    """
    rewards = np.zeros(num_episodes)
    goals_reached = 0

    for i in range(num_episodes):
        state = env.reset()[0]
        done = False
        truncated = False
        total_reward = 0

        while not (done or truncated):
            action = policy.get(state, env.action_space.sample())
            state, reward, done, truncated, _ = env.step(action)
            total_reward += reward

        rewards[i] = total_reward
        if done:
            goals_reached += 1

    # Calcular métricas principales
    average_reward = np.mean(rewards)
    goal_success_rate = (goals_reached / num_episodes) * 100

    # Mostrar métricas en tiempo real
    if episode is not None:
        print(f"[Episodio {episode}] Recompensa promedio: {average_reward:.2f}")
        print(f"[Episodio {episode}] Porcentaje de episodios exitosos: {goal_success_rate:.2f}%")
    else:
        print(f"Recompensa promedio: {average_reward:.2f}")
        print(f"Porcentaje de episodios exitosos: {goal_success_rate:.2f}%")

    # Acumular resultados si eval_results se proporciona
    if eval_results is not None:
        eval_results.setdefault('episodes', []).append(episode or len(eval_results.get('episodes', [])))
        eval_results.setdefault('average_rewards', []).append(average_reward)
        eval_results.setdefault('success_rates', []).append(goal_success_rate)

    return eval_results

def plot_evaluation_results(eval_results):
    """
    Genera las gráficas de las métricas acumuladas durante el entrenamiento.

    Args:
        eval_results (dict): Resultados acumulados de evaluaciones.
    """
    episodes = eval_results['episodes']
    average_rewards = eval_results['average_rewards']
    success_rates = eval_results['success_rates']

    print("\nResultados finales de evaluación:")
    for ep, reward, success in zip(episodes, average_rewards, success_rates):
        print(f"Episodio {ep}: Recompensa promedio = {reward:.2f}, Tasa de éxito = {success:.2f}%")

    plt.figure(figsize=(12, 5))

    # Gráfica de recompensas promedio
    plt.subplot(1, 2, 1)
    plt.plot(episodes, average_rewards, marker='o', linestyle='-')
    plt.title("Recompensas promedio por evaluación")
    plt.xlabel("Episodio de evaluación")
    plt.ylabel("Recompensa promedio")

    # Gráfica de tasas de éxito
    plt.subplot(1, 2, 2)
    plt.plot(episodes, success_rates, marker='o', linestyle='-')
    plt.title("Tasa de éxito por evaluación")
    plt.xlabel("Episodio de evaluación")
    plt.ylabel("Tasa de éxito (%)")

    plt.tight_layout()
    plt.show()


## Robot Pick and Place

### Renderizado

In [888]:
def render_optimal_policy(env, policy, max_steps=100, pause_time=0.5):
    """
    Renderiza paso a paso la ejecución de la política óptima en el entorno.

    Args:
        env: El entorno Gymnasium en el que se ejecutará la política.
        policy (dict): La política óptima, donde las claves son estados y los valores son acciones.
        max_steps (int): Número máximo de pasos para ejecutar el episodio.
        pause_time (float): Tiempo de pausa entre pasos para visualización (en segundos).

    Returns:
        dict: Resultados del episodio, incluyendo recompensa total y si el objetivo fue alcanzado.
    """
    # Reinicia el entorno y obtiene el estado inicial.
    state = env.reset()[0]
    env.render()

    total_reward = 0
    steps = 0
    goal_reached = False
    done = False
    truncated = False

    while steps < max_steps:
        # Obtiene la acción de la política para el estado actual.
        action = policy.get(state, env.action_space.sample())
        print(f"Estado: {state}, Acción seleccionada: {policy.get(state, 'Acción aleatoria')}")
        next_state, reward, done, truncated, _ = env.step(action)

        # Renderiza el entorno después de realizar la acción.
        env.render()
        total_reward += reward
        steps += 1

        if done or truncated:
            break

        state = next_state

        plt.pause(pause_time)

    env.close()
    plt.close()

    results = {
        "total_reward": total_reward,
        "steps": steps,
        "goal_reached": done
    }
    
    # Muestra un resumen de los resultados del episodio.
    print(f"Episodio terminado en {steps} pasos.")
    print(f"Recompensa total: {total_reward:.2f}")
    print(f"Objetivo alcanzado: {'Sí' if done else 'No'}")

    return results


### Entrenamiento y evaluación

##### Monte Carlo

In [889]:
# Elegir la función de recompensa a utilizar: calculate_reward_simple, calculate_reward_distance o calculate_reward_mixed
env = RobotPickAndPlaceEnv(reward_function=calculate_reward_simple)
num_episodes = 3000

In [890]:
print("Entrenando Monte Carlo...")
Q_mc, policy_mc = monte_carlo_es(env, num_episodes, graph=True)

Entrenando Monte Carlo...
[Episodio 1000] Recompensa promedio: -11674047900.00
[Episodio 1000] Porcentaje de episodios exitosos: 0.00%
[Episodio 2000] Recompensa promedio: -4027500000.00
[Episodio 2000] Porcentaje de episodios exitosos: 0.00%
[Episodio 3000] Recompensa promedio: -5143000000.00
[Episodio 3000] Porcentaje de episodios exitosos: 0.00%

Mostrando gráficas finales de evaluación...

Resultados finales de evaluación:
Episodio 1000: Recompensa promedio = -11674047900.00, Tasa de éxito = 0.00%
Episodio 2000: Recompensa promedio = -4027500000.00, Tasa de éxito = 0.00%
Episodio 3000: Recompensa promedio = -5143000000.00, Tasa de éxito = 0.00%


In [891]:
print("Visualizando la política óptima...")
render_optimal_policy(env, policy_mc, max_steps=100, pause_time=0.15)

Visualizando la política óptima...
Estado: 0, Acción seleccionada: 5
Estado: (0, 0, 1, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (1, 0, 1, 0, 5, 5, 5), Acción seleccionada: 0
Estado: (0, 0, 1, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (1, 0, 1, 0, 5, 5, 5), Acción seleccionada: 0
Estado: (0, 0, 1, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (1, 0, 1, 0, 5, 5, 5), Acción seleccionada: 0
Estado: (0, 0, 1, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (1, 0, 1, 0, 5, 5, 5), Acción seleccionada: 0
Estado: (0, 0, 1, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (1, 0, 1, 0, 5, 5, 5), Acción seleccionada: 0
Estado: (0, 0, 1, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (1, 0, 1, 0, 5, 5, 5), Acción seleccionada: 0
Estado: (0, 0, 1, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (1, 0, 1, 0, 5, 5, 5), Acción seleccionada: 0
Estado: (0, 0, 1, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (1, 0, 1, 0, 5, 5, 5), Acción seleccionada: 0
Estado: (0, 0, 1, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (1, 0

{'total_reward': -766800000, 'steps': 100, 'goal_reached': False}

##### Q-Learning

In [892]:
# Elegir la función de recompensa a utilizar: calculate_reward_simple, calculate_reward_distance o calculate_reward_mixed
env = RobotPickAndPlaceEnv(reward_function=calculate_reward_simple)
num_episodes = 3000

In [893]:
print("Entrenando Q-Learning...")
Q_ql, policy_ql = q_learning(env, num_episodes, alpha=0.1, gamma=0.9, epsilon=1.0, graph=True)

Entrenando Q-Learning...

Evaluación tras el episodio 1000:
[Episodio 1000] Recompensa promedio: -2953000000.00
[Episodio 1000] Porcentaje de episodios exitosos: 0.00%
Episodio 1000/3000 completado

Evaluación tras el episodio 2000:
[Episodio 2000] Recompensa promedio: -920470.00
[Episodio 2000] Porcentaje de episodios exitosos: 0.00%
Episodio 2000/3000 completado

Evaluación tras el episodio 3000:
[Episodio 3000] Recompensa promedio: -102590.00
[Episodio 3000] Porcentaje de episodios exitosos: 100.00%
Episodio 3000/3000 completado

Mostrando gráficas finales de evaluación...

Resultados finales de evaluación:
Episodio 1000: Recompensa promedio = -2953000000.00, Tasa de éxito = 0.00%
Episodio 2000: Recompensa promedio = -920470.00, Tasa de éxito = 0.00%
Episodio 3000: Recompensa promedio = -102590.00, Tasa de éxito = 100.00%


In [894]:
print("Visualizando la política óptima...")
render_optimal_policy(env, policy_ql, max_steps=100, pause_time=0.15)

Visualizando la política óptima...
Estado: 0, Acción seleccionada: 5
Estado: (0, 0, 1, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (1, 0, 1, 0, 5, 5, 5), Acción seleccionada: 3
Estado: (1, 1, 1, 0, 5, 5, 5), Acción seleccionada: 3
Estado: (1, 2, 1, 0, 5, 5, 5), Acción seleccionada: 5
Estado: (1, 2, 2, 0, 5, 5, 5), Acción seleccionada: 5
Estado: (1, 2, 3, 0, 5, 5, 5), Acción seleccionada: 3
Estado: (1, 3, 3, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (2, 3, 3, 0, 5, 5, 5), Acción seleccionada: 3
Estado: (2, 4, 3, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (3, 4, 3, 0, 5, 5, 5), Acción seleccionada: 5
Estado: (3, 4, 4, 0, 5, 5, 5), Acción seleccionada: 3
Estado: (3, 5, 4, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (4, 5, 4, 0, 5, 5, 5), Acción seleccionada: 1
Estado: (5, 5, 4, 0, 5, 5, 5), Acción seleccionada: 5
Estado: (5, 5, 5, 0, 5, 5, 5), Acción seleccionada: 6
Estado: (5, 5, 5, 1, -1, -1, -1), Acción seleccionada: 3
Estado: (5, 6, 5, 1, -1, -1, -1), Acción seleccionada: 3
Estado:

{'total_reward': -57680, 'steps': 27, 'goal_reached': True}

## Frozen Lake

### Entrenamiento y evaluación

In [895]:
def run_gymnasium_example():
    env = gymnasium.make('FrozenLake-v1', is_slippery=False)
    num_episodes = 5000

    print("Entrenando Monte Carlo en Frozen Lake...")
    Q_mc, policy_mc = monte_carlo_es(env, num_episodes)
    print('Política Monte Carlo en FrozenLake:', policy_mc)

    print("Entrenando Q-Learning en Frozen Lake...")
    Q_ql, policy_ql = q_learning(env, num_episodes, alpha=0.1, gamma=0.9, epsilon=1.0)
    print('Política Q-Learning en FrozenLake:', policy_ql)

# Probar Frozen Lake
run_gymnasium_example()

Entrenando Monte Carlo en Frozen Lake...
[Episodio 1000] Recompensa promedio: 1.00
[Episodio 1000] Porcentaje de episodios exitosos: 100.00%
[Episodio 2000] Recompensa promedio: 1.00
[Episodio 2000] Porcentaje de episodios exitosos: 100.00%
[Episodio 3000] Recompensa promedio: 1.00
[Episodio 3000] Porcentaje de episodios exitosos: 100.00%
[Episodio 4000] Recompensa promedio: 1.00
[Episodio 4000] Porcentaje de episodios exitosos: 100.00%
[Episodio 5000] Recompensa promedio: 1.00
[Episodio 5000] Porcentaje de episodios exitosos: 100.00%

Mostrando gráficas finales de evaluación...

Resultados finales de evaluación:
Episodio 1000: Recompensa promedio = 1.00, Tasa de éxito = 100.00%
Episodio 2000: Recompensa promedio = 1.00, Tasa de éxito = 100.00%
Episodio 3000: Recompensa promedio = 1.00, Tasa de éxito = 100.00%
Episodio 4000: Recompensa promedio = 1.00, Tasa de éxito = 100.00%
Episodio 5000: Recompensa promedio = 1.00, Tasa de éxito = 100.00%
Política Monte Carlo en FrozenLake: {8: 2, 4

## Conclusiones

Una vez ejecutados ambos algoritmos con las 3 funciones de recompensa, sacamos las siguientes conclusiones:

- El algoritmo de Monte Carlo no funciona correctamente con niguna de las funciones. No es un algoritmo que funcione  bien en entornos complejos y con funciones de recompensa poco claras.

- Q-Learning converge y cumple con el objetivo usando las 3 funciones. Muestra su adaptabilidad en entornos más complejos gracias a la actualización paso a paso y su capacidad para trabajar con recompensas intermedias.

- Ambos algoritmos funcionan bien en el entorno de Frozen Lake, por lo que confirmamos que son efectivos en problemas discretos más simples y recompensas bien definidas.

### calculate_reward_simple

#### Monte Carlo

Con la función simple, el robot practicamente no se mueve del punto de origen y entra en bucle. Esto puede ser porque el algoritmo da una política exploratoria inicial y la función de recompensa no incita al robot a explorar en dirección hacia la pieza con recompensas intermedias suficientes.

De la gráfica podemos decir que el robot no aprende nada y está estancado en recompensas negativas. No hay indicios de posible mejora en una ejecución con mayor número de episodios.

![Recompensa acumulada en Monte Carlo con la función de recompensa simple](./monte_carlo_accumulated_rewards_calculate_reward_simple.png)

#### Q-Learning

Con Q-Learning, la función simple entrena al robot para llegar al objetivo. Si observamos la gráfica de recompensa acumulada vemos que:

- La recompensa acumulada empieza con valores negativos, lo que indica exploración inicial y penalizaciones. También hay variaciones, estas pueden ser por la tasa de exploración.

- Las variaciones van disminuyendo y se empiezan a estabilizar, lo que indica que el robot está aprendiendo una buena política basada en las recompensas obtenidas.

![Recompensa acumulada en Q-Learning con la función de recompensa simple](./q_learning_accumulated_rewards_calculate_reward_simple.png)

### calculate_reward_distance

#### Monte Carlo

Con esta función, Monte Carlo no funciona bien ya que, después de coger la pieza, entra en un bucle. Esto puede deberse a que el algoritmo en sí funciona con una política inicial aleatoria y, si nada le hace salir del bucle, se queda ahí. También, es posible que el mal rendimiento que proporciona sea porque limitamos el número de pasos a 1000, por falta de capacidad de computación, y no aprende lo suficiente.

Observamos en la gráfica que, sin embargo, la recompensa acumulada por episodio es positiva y alta. Esto se debe a que con esta función no penalizamos y la recompensa es siempre proporcional a la distancia del objetivo, ya sea coger la pieza o soltarla.

![Recompensa acumulada en Monte Carlo con la función de recompensa de distancia](./monte_carlo_accumulated_rewards_calculate_reward_distance.png)

#### Q-Learning

Con la función de recompensa basada en la distancia, Q-Learning converge y suelta la pieza en el objetivo. Observamos como evoluciona la recompensa acumulada a lo largo de los episodios de entrenamiento:

- Durante la primera mitad de los episodios, la recompensa acumulada muestra una tendencia creciente con variabilidad, indicando que el robot está aprendiendo a realizar acciones más efectivas, logrando acercarse al objetivo y completar la tarea con mayor frecuencia.

- En los isguientes episodios, la recompensa acumulada se empieza a estabilizar, lo que sugiere que el robot ha aprendido una política buena y está ejecutando las acciones necesarias para cumplir el objetivo de manera eficiente.

- Hacia el final de los episodios, hay cierta variabilidad en las recompensas, puede deberse a que el robot siga explorando posibles rutas.

![Recompensa acumulada en Q-Learning con la función de recompensa de distancia](./q_learning_accumulated_rewards_calculate_reward_distance.png)

### calculate_reward_mixed

#### Monte Carlo

Usando la función mixed, Monte Carlo no consigue que el robot llegue a coger la pieza. Mirando la gráfica, observamos como las recompensas son bastante negativas, lo que nos deja ver que la mayoría de acciones realizadas por el robot han sido penalizadas.

Tampoco podemos observar que haya indicios de aprendizaje ya que, aunque hay variaciones, el valor de la recompensa acumulada es bastante constante, no hay tendencia positiva.

Estos resultados pueden deberse a que Monte Carlo depende de la exploración para encontrar políticas efectivas y si la función no es buena, los intentos aleatorios iniciales no logran guiar al robot y se puede quedar atrapado en una política mala. Además, Monte Carlo suele necesitar más episodios para converger que Q-Learning.

![Recompensa acumulada en Monte Carlo con la función de recompensa mixed](./monte_carlo_accumulated_rewards_calculate_reward_mixed.png)

#### Q-Learning

Con esta función, Q-Learning entrena al robot y consigue llegar al objetivo. Analizando la gráfica vemos:

- Al principio, las recompensas acumuladas son negativas, probablemente porque la función de recompensa aplica penalizaciones altas por errores y movimientos redundantes.

- Durante el segundo período, entre los 1000 y 2000 episodios, las recompensas acumuladas comienzan a aumentar. Parece que el robot comienza a encontrar estrategias más efectivas para minimizar las penalizaciones y maximizar las recompensas.

- En los últimos episodios, las recompensas se estabilizan, con algunas variaciones. Esto indica que el robot ha aprendido una política buena para cumplir el objetivo.

![Recompensa acumulada en Q-Learning con la función de recompensa mixed](./q_learning_accumulated_rewards_calculate_reward_mixed.png)